In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import layers

from modules.dataset import OneHotProtDataset
from sklearn.model_selection import train_test_split

from modules.model.VAE import VAE

2024-03-23 06:53:27.684866: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 06:53:27.684930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 06:53:27.685653: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 06:53:27.692088: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
x_data, y_data, vector_size, alphabet_size = OneHotProtDataset.get_data()
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42
)

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


### Encoder

In [4]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(vector_size, alphabet_size, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

2024-03-23 06:53:33.186559: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 06:53:33.278093: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 06:53:33.280878: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1500, 22, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 750, 11, 32)          320       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 375, 6, 64)           18496     ['conv2d[0][0]']              
                                                                                                  
 flatten (Flatten)           (None, 144000)               0         ['conv2d_1[0][0]']            
                                                                                            

### Decoder

In [5]:
# Define decoder model. should have output shape (vector_size, alphabet_size, 1)

inputs  = keras.Input(shape=(latent_dim,))
x       = layers.Dense(vector_size // 2 * alphabet_size // 2 * 64, activation="relu")(inputs)
x       = layers.Reshape((vector_size // 2, alphabet_size // 2, 64))(x)
x       = layers.Conv2DTranspose(64, 3, strides=1, padding="same", activation="relu")(x)
x       = layers.Conv2DTranspose(64, 3, strides=2, padding="same", activation="relu")(x)
# x       = layers.Conv2DTranspose(32, 3, strides=2, padding="same", activation="relu")(x)
outputs = layers.Conv2DTranspose(1,  3, padding="same", activation="sigmoid")(x)

decoder = keras.Model(inputs, outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 528000)            1584000   
                                                                 
 reshape (Reshape)           (None, 750, 11, 64)       0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 750, 11, 64)       36928     
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 1500, 22, 64)      36928     
 Transpose)                                                      
                                                                 
 conv2d_transpose_2 (Conv2D  (None, 1500, 22, 1)       577       
 Transpose)                                                      
                                                                 
Total para

## Train the VAE

In [6]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
dataset = np.concatenate([x_train, x_test], axis=0)

In [7]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(dataset, epochs=30, batch_size=128)


2024-03-23 06:53:40.980446: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 400356000 exceeds 10% of free system memory.
2024-03-23 06:53:41.226426: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 400356000 exceeds 10% of free system memory.


Epoch 1/30


2024-03-23 06:53:42.682401: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-03-23 06:53:47.561970: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fd20cf22250 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-23 06:53:47.562012: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-03-23 06:53:47.567053: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711176827.636619   34837 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


24/24 [==============================] - 26s 574ms/step - loss: 16363.1158 - reconstruction_loss: 8257.5029 - kl_loss: 1266.7114
Epoch 2/30
24/24 [==============================] - 8s 334ms/step - loss: 1353.0337 - reconstruction_loss: 1278.0284 - kl_loss: 11.5341
Epoch 3/30
24/24 [==============================] - 8s 338ms/step - loss: 1107.1473 - reconstruction_loss: 1097.0563 - kl_loss: 6.5989
Epoch 4/30
24/24 [==============================] - 8s 341ms/step - loss: 1053.9423 - reconstruction_loss: 1028.5475 - kl_loss: 6.1347
Epoch 5/30
24/24 [==============================] - 8s 343ms/step - loss: 1010.9942 - reconstruction_loss: 995.2991 - kl_loss: 6.1588
Epoch 6/30
24/24 [==============================] - 8s 347ms/step - loss: 1011.3631 - reconstruction_loss: 975.8294 - kl_loss: 6.1376
Epoch 7/30
24/24 [==============================] - 8s 352ms/step - loss: 961.7528 - reconstruction_loss: 957.6442 - kl_loss: 6.0674
Epoch 8/30
24/24 [==============================] - 9s 355ms/ste

## References

- <https://keras.io/examples/generative/vae/>